<a href="https://colab.research.google.com/github/Anthei0774/Advent-of-Code-2021/blob/main/Day_14_Extended_Polymerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://adventofcode.com/2021/day/14

In [1]:
puzzle = '''NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C'''

# with open('14.txt') as f:
#     puzzle = f.read()

################################################################################

puzzle = puzzle.split('\n')

polymer_template = puzzle[0]
print('Polymer template:', polymer_template)

pairs = { l.split(' ')[0] : l.split(' ')[2] for l in puzzle[2:] }
pairs

Polymer template: NNCB


{'BB': 'N',
 'BC': 'B',
 'BH': 'H',
 'BN': 'B',
 'CB': 'H',
 'CC': 'N',
 'CH': 'B',
 'CN': 'C',
 'HB': 'C',
 'HC': 'B',
 'HH': 'N',
 'HN': 'C',
 'NB': 'B',
 'NC': 'B',
 'NH': 'C',
 'NN': 'C'}

# Part 2

Realize that for a given target, not every previous step should be calculated. Suppose, you have calculated what happens with every pair after two iterations, e.g.:
- BB -> BNB -> BBNBB
- BN -> BBN -> BNBBN
- NB -> NBB -> NBBNB

To get to 4, it is enough to look up what happens with a pair after 2 iterations. With an example 4 steps can be calculated by iterating from 2 to 3, then from 3 to 4, or executing a "double" iteration at step 2.
- BBNBB -> BNBBNBBNB -> BBNBBNBBNBBNBBNBB --- this is going sequentially
- BBNBB -> (BBNBB) + (BNBBN) + (NBBNB) + (BBNBB) = BBNBBNBBNBBNBBNBB --- this is recalling the 2nd step of every pair, and then concatenating them, appropriately leaving out some of the inner elements.

With this, we can greately reduce the number of steps that needs to be computed. For 40 steps, only the following needs to be precomputed:
- 40 -> 20
- 20 -> 10
- 10 -> 5
- 5 -> 3, 2
- 3 -> 2, 1
- 2 -> 1
- 1

In [2]:
# set target, and calculate steps needed
final_step = 10 # part 1 - previously coded brute force
final_step = 40 # part 2 - need to use my brainz
steps_recursive = {1 : []}

# calculate "major" steps, halving substep s until reaching 1
s = final_step
while s != 1:
    steps_recursive[s] = sorted(list(set([s // 2, s - s // 2])))
    s //= 2

# fill the "gaps" between, any substeps that have been left out
while True:

    # add new substep if it is needed, but not in container
    new_sr = {}
    for sr in steps_recursive:
        for s in steps_recursive[sr]:
            if s not in steps_recursive:
                new_sr[s] = sorted(list({s // 2, s - s // 2}))

    # if no new substeps, break out, else append
    if new_sr == {}:
        break
    else:
        for s in new_sr: steps_recursive[s] = new_sr[s]

################################################################################

steps_recursive

{1: [], 2: [1], 3: [1, 2], 5: [2, 3], 10: [5]}

In [4]:
iterations = {p: {1: p[0] + pairs[p] + p[1]} for p in pairs}

# iterate through all but the last value
# only those which need to be precomputed
sr_sorted_keys = sorted(list(steps_recursive))
for i in range(1, len(sr_sorted_keys) - 1):

    # s0 and s1 stores the one / two predecessor steps
    # if sr = 4, then s0 = s1 = 2
    # if sr = 5, then s0 = 2, s1 = 3
    sr = sr_sorted_keys[i]
    steps = steps_recursive[sr]
    s0 = 0
    s1 = 0
    if len(steps) == 1:
        s0 = steps[0]
        s1 = steps[0]
    else:
        s0 = steps[0]
        s1 = steps[1]

    # for each pair, execute the "multi-step"
    for p in iterations:

        # initialize new iterated string = it
        # this will be at index sr
        it = ''

        # take the given pair's s0 iterated string = it1
        it1 = iterations[p][s0]

        # for each pair in it1
        for x in range(0, len(it1) - 1):
            it2 = it1[x : x + 2]

            # take the other pair's s1 iterated string = it2
            # and append it appropriately to the final version
            # appropriately = remove last char except in the last pair
            if x != len(it1) - 2:
                it += iterations[it2][s1][: -1]
            else:
                it += iterations[it2][s1]

        iterations[p][sr] = it

################################################################################

iterations

{'BB': {1: 'BNB',
  2: 'BBNBB',
  3: 'BNBBNBBNB',
  5: 'BNBBNBBNBBNBBNBBNBBNBBNBBNBBNBBNB'},
 'BC': {1: 'BBC',
  2: 'BNBBC',
  3: 'BBNBBNBBC',
  5: 'BBNBBNBBNBBNBBNBBNBBNBBNBBNBBNBBC'},
 'BH': {1: 'BHH',
  2: 'BHHNH',
  3: 'BHHNHCNCH',
  5: 'BHHNHCNCHBCCNBCBHCBBCNCCNBBBCHBHH'},
 'BN': {1: 'BBN',
  2: 'BNBBN',
  3: 'BBNBBNBBN',
  5: 'BBNBBNBBNBBNBBNBBNBBNBBNBBNBBNBBN'},
 'CB': {1: 'CHB',
  2: 'CBHCB',
  3: 'CHBHHBCHB',
  5: 'CHBHHBCHBHHNHCNCHBCHBNBBCHBHHBCHB'},
 'CC': {1: 'CNC',
  2: 'CCNBC',
  3: 'CNCCNBBBC',
  5: 'CNCCNBBBCCNBCNCCNBBNBBNBBBNBBNBBC'},
 'CH': {1: 'CBH',
  2: 'CHBHH',
  3: 'CBHCBHHNH',
  5: 'CBHCBHHNHCBBCBHCBHHNHCNCHBCCNBCBH'},
 'CN': {1: 'CCN',
  2: 'CNCCN',
  3: 'CCNBCNCCN',
  5: 'CCNBCNCCNBBNBNBBCNCCNBBBCCNBCNCCN'},
 'HB': {1: 'HCB',
  2: 'HBCHB',
  3: 'HCBBCBHCB',
  5: 'HCBBCBHCBBNBBNBBCBHCBHHNHCBBCBHCB'},
 'HC': {1: 'HBC',
  2: 'HCBBC',
  3: 'HBCHBNBBC',
  5: 'HBCHBNBBCHBHHBCHBNBBNBBNBBNBBNBBC'},
 'HH': {1: 'HNH',
  2: 'HCNCH',
  3: 'HBCCNBCBH',
  5: 'HBCHBNBBCCNBCN

In [5]:
# for each iteration, calculate their char-counts
counts = {pair: iterations[pair].copy() for pair in iterations}

for pair in counts:
    for iter in counts[pair]:
        
        string = counts[pair][iter]
        abc = set(string)
        counts[pair][iter] = {char: string.count(char) for char in abc}

################################################################################

counts

{'BB': {1: {'B': 2, 'N': 1},
  2: {'B': 4, 'N': 1},
  3: {'B': 6, 'N': 3},
  5: {'B': 22, 'N': 11}},
 'BC': {1: {'B': 2, 'C': 1},
  2: {'B': 3, 'C': 1, 'N': 1},
  3: {'B': 6, 'C': 1, 'N': 2},
  5: {'B': 22, 'C': 1, 'N': 10}},
 'BH': {1: {'B': 1, 'H': 2},
  2: {'B': 1, 'H': 3, 'N': 1},
  3: {'B': 1, 'C': 2, 'H': 4, 'N': 2},
  5: {'B': 10, 'C': 10, 'H': 8, 'N': 5}},
 'BN': {1: {'B': 2, 'N': 1},
  2: {'B': 3, 'N': 2},
  3: {'B': 6, 'N': 3},
  5: {'B': 22, 'N': 11}},
 'CB': {1: {'B': 1, 'C': 1, 'H': 1},
  2: {'B': 2, 'C': 2, 'H': 1},
  3: {'B': 3, 'C': 2, 'H': 4},
  5: {'B': 10, 'C': 7, 'H': 13, 'N': 3}},
 'CC': {1: {'C': 2, 'N': 1},
  2: {'B': 1, 'C': 3, 'N': 1},
  3: {'B': 3, 'C': 4, 'N': 2},
  5: {'B': 15, 'C': 9, 'N': 9}},
 'CH': {1: {'B': 1, 'C': 1, 'H': 1},
  2: {'B': 1, 'C': 1, 'H': 3},
  3: {'B': 2, 'C': 2, 'H': 4, 'N': 1},
  5: {'B': 9, 'C': 10, 'H': 10, 'N': 4}},
 'CN': {1: {'C': 2, 'N': 1},
  2: {'C': 3, 'N': 2},
  3: {'B': 1, 'C': 5, 'N': 3},
  5: {'B': 10, 'C': 13, 'N': 10}},


In [6]:
# finalize: get the two parts of the final step
# step0 + step1 = fs
fs = steps_recursive[final_step]

step0 = 0
step1 = 0
if len(fs) == 1:
    step0 = fs[0]
    step1 = fs[0]
else:
    step0 = fs[0]
    step1 = fs[1]

# initialize a final counter
final_counter = {}

# for each pair in the polymer template
for i in range(0, len(polymer_template) - 1):
    pair = polymer_template[i : i + 2]
    
    # select the step0 iteration of the pair
    iter0 = iterations[pair][step0]

    # for each pair in the step0 iteration
    for j in range(0, len(iter0) - 1):
        iter0_pair = iter0[j : j + 2]

        # select the counts of the step1 for the pair
        counts1 = counts[iter0_pair][step1]

        # for each count, increase the final counter
        for char in counts1:
            if char not in final_counter:
                final_counter[char] = counts1[char]
            else:
                final_counter[char] += counts1[char]
        
        # watch out for concatenation, avoid counting twice
        # solving by removing the very last char in the end
        last_char = iter0_pair[-1]
        final_counter[last_char] -= 1

# add the polymer template's last char to the counter
last_char = polymer_template[-1]
final_counter[last_char] += 1

################################################################################

final_counter

{'B': 1749, 'C': 298, 'H': 161, 'N': 865}

In [7]:
# get min-max values and enjoy
m = min(final_counter.values())
M = max(final_counter.values())

M - m

1588